In [ ]:
use project  #have used my project databases

In [ ]:
CREATE TABLE Books (
    book_id INT PRIMARY KEY,
    title VARCHAR(255),
    author VARCHAR(255),
    genre VARCHAR(100),
    publication_date DATE,
    isbn VARCHAR(20), -- Adjust the size as necessary to accommodate your ISBNs
    quantity_available INT
);


In [ ]:
INSERT INTO Books (book_id, title, author, genre, publication_date, isbn, quantity_available)
VALUES
    (1, 'Harry Potter', 'J.K. Rowling', 'Fantasy', '1997-06-26', '9788700631625', 5),
    (2, 'To Kill a Mockingbird', 'Harper Lee', 'Fiction', '1960-07-11', '9780061120084', 3),
    (3, 'The Great Gatsby', 'F. Scott Fitzgerald', 'Classic', '1925-04-10', '9780743273565', 2);


In [ ]:
create table Authors(
author_id int Primary Key,
author_name varchar(100));

In [ ]:
INSERT INTO Authors (author_id, author_name)
VALUES
    (1, 'J.K. Rowling'),
    (2, 'Harper Lee'),
    (3, 'F. Scott Fitzgerald');


In [ ]:
create table Members(
member_id int Primary Key,
member_name varchar(100),
member_email varchar(100),
member_phone varchar(20));

In [ ]:
INSERT INTO Members (member_id, member_name, member_email, member_phone)
VALUES
    (1, 'John Doe', 'john.doe@example.com', '123-456-7890'),
    (2, 'Jane Smith', 'jane.smith@example.com', '987-654-3210');


In [ ]:
CREATE TABLE Borrowings (
    borrowing_id INT PRIMARY KEY,
    book_id INT,
    member_id INT,
    borrowing_date DATE,
    return_date DATE,
    is_returned BOOLEAN,
    FOREIGN KEY (book_id) REFERENCES Books(book_id),
    FOREIGN KEY (member_id) REFERENCES Members(member_id)
);


In [ ]:
INSERT INTO Borrowings (borrowing_id, book_id, member_id, borrowing_date, return_date, is_returned)
VALUES
    (1, 1, 1, '2024-02-10', '2024-02-20', true),
    (2, 2, 2, '2024-02-15', NULL, false);


In [ ]:
create table Publishers(
publisher_id int Primary Key,
publisher_name varchar(100),
publisher_country varchar(100));

In [ ]:
INSERT INTO Publishers (publisher_id, publisher_name, publisher_country)
VALUES
    (1, 'Penguin Random House', 'United States'),
    (2, 'HarperCollins', 'United Kingdom');


In [ ]:
CREATE TABLE BookCopies (
    copy_id INT PRIMARY KEY,
    book_id INT,
    copy_number INT,
    copy_condition VARCHAR(100),
    shelf_location VARCHAR(100),
    FOREIGN KEY (book_id) REFERENCES Books(book_id)
);


In [ ]:
INSERT INTO BookCopies (copy_id, book_id, copy_number, copy_condition, shelf_location)
VALUES
    (1, 1, '001', 'Good', 'A1'),
    (2, 1, '002', 'Fair', 'B3');


In [ ]:
CREATE TABLE AuthorsBooksMapping (
    author_book_id INT PRIMARY KEY,
    author_id INT,
    book_id INT,
    FOREIGN KEY (author_id) REFERENCES Authors(author_id),
    FOREIGN KEY (book_id) REFERENCES Books(book_id)
);


In [ ]:
INSERT INTO AuthorsBooksMapping (author_book_id, author_id, book_id)
VALUES
    (1, 1, 1),
    (2, 2, 2);


In [ ]:
CREATE TABLE Reviews (
    review_id INT PRIMARY KEY,
    book_id INT,
    member_id INT,
    rating DECIMAL(3,1),
    review_text TEXT,
    review_date DATE,
    FOREIGN KEY (book_id) REFERENCES Books(book_id),
    FOREIGN KEY (member_id) REFERENCES Members(member_id)
);


In [ ]:
INSERT INTO Reviews (review_id, book_id, member_id, rating, review_text, review_date)
VALUES
    (1, 1, 1, 4.5, 'A classic masterpiece.', '2024-02-12'),
    (2, 2, 2, 5.0, 'Absolutely loved it!', '2024-02-18');


In [ ]:
CREATE TABLE Transactions (
    transaction_id INT PRIMARY KEY,
    member_id INT,
    transaction_date DATE,
    transaction_type VARCHAR(20),
    amount_paid DECIMAL(10, 2),
    FOREIGN KEY (member_id) REFERENCES Members(member_id)
);


In [ ]:
INSERT INTO Transactions (transaction_id, member_id, transaction_date, transaction_type, amount_paid)
VALUES
    (1, 1, '2024-02-10', 'Borrow', 0),
    (2, 2, '2024-02-15', 'Borrow', 0);


In [ ]:
#1.List all books borrowed by a specific member?

SELECT b.title, b.author, b.genre, b.publication_date, b.isbn
FROM Books b
JOIN Borrowings bor ON b.book_id = bor.book_id
JOIN Members m ON bor.member_id = m.member_id
WHERE m.member_name = 'John Doe';


In [ ]:
#2.Find the most popular genres?

SELECT b.genre, COUNT(*) AS borrow_count
FROM Borrowings bor
JOIN Books b ON bor.book_id = b.book_id
GROUP BY b.genre
ORDER BY borrow_count DESC;   #this gives the result as fantasy the most popular genre


In [ ]:
#3.Identify books with the highest average rating?
SELECT b.title, b.author, AVG(r.rating) AS avg_rating
FROM Books b
JOIN Reviews r ON b.book_id = r.book_id
GROUP BY b.title, b.author
ORDER BY avg_rating DESC;


In [ ]:
#4.List all members who have borrowed more than 5 books?

SELECT m.member_id, m.member_name, COUNT(*) AS borrow_count
FROM Members m
JOIN Borrowings bor ON m.member_id = bor.member_id
GROUP BY m.member_id, m.member_name
HAVING COUNT(*) > 5;


In [ ]:
#5.List all members who have borrowed less than 5 books?
SELECT m.member_id, m.member_name, COUNT(*) AS borrow_count
FROM Members m
JOIN Borrowings bor ON m.member_id = bor.member_id
GROUP BY m.member_id, m.member_name
HAVING COUNT(*) < 5;


In [ ]:
#6. Retrieve the top-rated books with at least 5 reviews?

SELECT title, author, avg_rating
FROM (
    SELECT b.title, b.author, AVG(r.rating) AS avg_rating, COUNT(r.review_id) AS review_count
    FROM Books b
    JOIN Reviews r ON b.book_id = r.book_id
    GROUP BY b.title, b.author
    HAVING COUNT(r.review_id) >= 5
) AS top_books
ORDER BY avg_rating DESC;


In [ ]:
#7.Calculate the total revenue generated from book purchases?
SELECT SUM(amount_paid) AS total_revenue
FROM Transactions
WHERE transaction_type = 'purchase';


In [ ]:
#8.List all books with their respective authors and publishers:
SELECT 
    b.title AS book_title,
    b.author AS book_author
FROM 
    Books b
JOIN 
    Authors a ON b.author = a.author_name;



In [ ]:
#9.Find books that are currently available for borrowing?

SELECT *
FROM Books
WHERE quantity_available > 0;


In [ ]:
#10.Identify members who have overdue books?
SELECT m.member_id, m.member_name
FROM Members m
JOIN Borrowings b ON m.member_id = b.member_id
WHERE b.return_date < CURRENT_DATE AND b.is_returned = false;


In [ ]:
#11.List the top 10 most borrowed books?

SELECT b.title AS book_title, COUNT(*) AS borrow_count
FROM Books b
JOIN Borrowings bor ON b.book_id = bor.book_id
GROUP BY b.title
ORDER BY borrow_count DESC
LIMIT 10;


In [ ]:
#12.Calculate the average number of days a book is borrowed for?
SELECT AVG(DATEDIFF(return_date, borrowing_date)) AS avg_days_borrowed
FROM Borrowings
WHERE is_returned = true;


In [ ]:
#13.Find the total number of books published in each year?
SELECT YEAR(publication_date) AS publication_year, COUNT(*) AS total_books_published
FROM Books
GROUP BY publication_year
ORDER BY publication_year;


In [ ]:
#14.Identify members who have borrowed books more than once?
SELECT member_id, member_name
FROM Members
WHERE member_id IN (
    SELECT member_id
    FROM Borrowings
    GROUP BY member_id
    HAVING COUNT(*) > 1
);


In [ ]:
#15.List all books with their respective authors and average ratings?
SELECT 
    b.title AS book_title, 
    a.author_name AS author, 
    AVG(r.rating) AS average_rating
FROM 
    Books b
JOIN 
    Authors a ON b.author = a.author_name
LEFT JOIN 
    Reviews r ON b.book_id = r.book_id
GROUP BY 
    b.title, a.author_name;


In [ ]:
#16.Calculate the total number of copies available for each book?
SELECT 
    b.title AS book_title, 
    COUNT(bc.copy_id) AS total_copies_available
FROM 
    Books b
JOIN 
    BookCopies bc ON b.book_id = bc.book_id
GROUP BY 
    b.title;


In [ ]:
#17.Create a view of transaction table and provide privilege to another user. The user can view only member id and transaction date and privilege 
#should be to select id who made transaction on any specific date.

Step 1: Create the view:
CREATE VIEW LimitedTransactionView AS
SELECT member_id, transaction_date
FROM Transactions;


Step 2(Optional): Create a stored procedure to select records for a specific date:


DELIMITER //

CREATE PROCEDURE SelectTransactionsByDate(IN search_date DATE)
BEGIN
    SELECT member_id
    FROM LimitedTransactionView
    WHERE transaction_date = search_date;
END //

DELIMITER ;

